In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

In [2]:
#3d camera data example
orig = np.array([-174.08, -67.02, 400.86])
x100 = np.array([-74.71, -61.55, 402.18])
y100 = np.array([-179.10, 32.16, 399.36])
z100 = np.array([-177.71, -67.53, 498.95])

a1 = x100 - orig
print(a1)
a1 = a1/100
print(a1)


a2 = y100 - orig
print(a2)
a2 = a2/100
print(a2)

a3 = z100 - orig
print(a3)
a3 = a3/100
print(a3)

a = -174
b = -67
c = 400

a4 = orig - (a*a1+b*a2+c*a3)
print(a4)

a1 = np.concatenate((a1,[0]))
a2 = np.concatenate((a2,[0]))
a3 = np.concatenate((a3,[0]))
a4 = np.concatenate((a4,[1]))
t_mat = np.stack((a1,a2,a3,a4),axis = 1)
np.round(t_mat,2)
print(t_mat)

[99.37  5.47  1.32]
[0.9937 0.0547 0.0132]
[-5.02 99.18 -1.5 ]
[-0.0502  0.9918 -0.015 ]
[-3.63 -0.51 98.09]
[-0.0363 -0.0051  0.9809]
[ 9.9804 10.9884  9.7918]
[[ 0.9937 -0.0502 -0.0363  9.9804]
 [ 0.0547  0.9918 -0.0051 10.9884]
 [ 0.0132 -0.015   0.9809  9.7918]
 [ 0.      0.      0.      1.    ]]


In [3]:
# Transformation Matrix Code

def trans_mat(abc,orig,x_move,x_moved_pos,y_move,y_moved_pos,z_move,z_moved_pos):
    a1 = x_moved_pos - orig
    a1 = a1/x_move
    a2 = y_moved_pos - orig
    a2 = a2/y_move
    a3 = z_moved_pos - orig
    a3 = a3/z_move
    abc = list(abc)
    a4 = orig - (abc[0]*a1+abc[1]*a2+abc[2]*a3)

    #rotation matrix orthogonality check
    print(np.sum(np.power(a1,2)))
    print(np.sum(np.power(a2,2)))
    print(np.sum(np.power(a3,2)))

    a1 = np.concatenate((a1,[0]))
    a2 = np.concatenate((a2,[0]))
    a3 = np.concatenate((a3,[0]))
    a4 = np.concatenate((a4,[1]))
    t_mat = np.stack((a1,a2,a3,a4),axis = 1)
    #print(t_mat)

    return t_mat

In [4]:
trans_mat((-174,-67,400), orig,100,x100,100,y100,100,z100)

0.9906060200000003
0.9864122799999998
0.9635085099999995


array([[ 0.9937, -0.0502, -0.0363,  9.9804],
       [ 0.0547,  0.9918, -0.0051, 10.9884],
       [ 0.0132, -0.015 ,  0.9809,  9.7918],
       [ 0.    ,  0.    ,  0.    ,  1.    ]])

In [5]:
def check_orthogonal(rot_mat):
    print("Orthogonality Check for Rotation Matrix")
    for i in range(3):
        print((np.sum(np.power(rot_mat[i],2))))

In [6]:
def rot_mat_create(mu, sigma):
    alpha,beta,gamma = np.random.normal(mu, sigma, 3)
    print("alpha, beta, gamma", alpha,beta,gamma)
    rx = [[1,0,0],[0,np.cos(alpha),np.sin(alpha)],[0,-np.sin(alpha),np.cos(alpha)]]
    ry = [[np.cos(beta),0,-np.sin(beta)],[0,1,0],[np.sin(beta),0,np.cos(beta)]]
    rz = [[np.cos(gamma),np.sin(gamma),0],[-np.sin(gamma),np.cos(gamma),0],[0,0,1]]
    #create rotation matrix
    rot_mat = np.dot(np.dot(rx,ry),rz)
    #check_orthogonal(rot_mat)

    return rot_mat
#print(rot_mat_create(1,10000))

In [7]:
def trans_mat_create(mu, sigma,translation):
    #translation 4*1 array like [1,1,1,1]
    alpha,beta,gamma = np.random.normal(mu, sigma, 3)

    #print(alpha,beta,gamma)
    rx = [[1,0,0],[0,np.cos(alpha),np.sin(alpha)],[0,-np.sin(alpha),np.cos(alpha)]]
    ry = [[np.cos(beta),0,-np.sin(alpha)],[0,1,0],[np.sin(beta),0,np.cos(beta)]]
    rz = [[np.cos(gamma),np.sin(gamma),0],[-np.sin(gamma),np.cos(gamma),0],[0,0,1]]
    #create rotation matrix
    rot_mat = np.dot(np.dot(rx,ry),rz)
    check_orthogonal(rot_mat)

    rot_mat = np.vstack([rot_mat, [0, 0, 0]])
    print(rot_mat)
    #create translation matrix
    trans_mat = np.hstack([rot_mat, np.array(translation).reshape(-1, 1)])
    print(trans_mat)

    return trans_mat

#trans_mat_create(1,0.1,[1,1,1,1])

# **Shoe Box Application**

In [8]:
#abc
abc = np.array([333, 210, 0])
orig = np.array([131.907928,-38.184845,695.467651])
#x =-50
x = np.array([169.025024,-63.640034,677.320190])
#y =-50
y = np.array([100.909630,-63.721424,667.450317])
#z = 50
z = np.array([137.838013,-2.953046,661.316284])
#p
p = np.array([227.846466,-110.801971,642.048706,1])

In [9]:
#Error Evaluation
T = trans_mat(abc, orig, -50, x, -50, y, 50, z)
T_inv = np.linalg.inv(T)
res = np.dot(T_inv, p)

print(res)
error = abs((np.array([200,200,0,1]) - res))
error = error[0:2]

error_per = 100*error/[200,200]
print(error_per)

0.9419903212981836
0.9591929401550403
0.9771045747409275
[197.58735917 202.57517471   0.16186238   1.        ]
[1.20632041 1.28758735]
